<a href="https://colab.research.google.com/github/betoval/learning-tensorflow/blob/master/intro-tensorflow2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

var = tf.Variable([3,3])

#the command mentioned in the book is deprecated
#config.list is the new command
if tf.config.list_physical_devices('GPU'):
  print('Running on GPU')
  print('GPU #0?')
  print(var.device.endswith('GPU:0'))
else:
  print('Running on CPU')

Running on CPU


If we choose GPU in the notebook settings, then we will obtain GPU as output (duh). However, we do not need a lot of resources right now, we will stick to CPU.

**Declaring eager variables, wtf is a tensor?, and all that preliminary stuff** 

The rank specifies the number of "dimensions" (n) of the tensor, that is, it indicates the number of indices that are required to specify a particular element of that tensor. For me, it is easier to think of tensors as n-forms. In the example below, we are talking about a 3-form, that is, an object with 3 indices.
If you want to visualize them, then think of tensors as matrices. In this example, we are working with a 3 dimensional matrix with 2 slices, 2 rows and 3 columns (this info is given by the shape of the tensor, that is, shape = (2, 2, 3)). Thus, if we want to assign a new value to an entry, then we need to specify three numbers corresponding to the slice, row and column.

In [0]:
#declaring eager variables
t0 = 24 #python variable
t1 = tf.Variable(42) # scalar or 0 rank tensor
#3-form or rank 3 tensor
t2 = tf.Variable([[[0.,1.,2.],[3.,4.,5.]],[[6.,7.,8.],[9.,10.,11.]]]) #datatype=float32
print(tf.rank(t2)) #print the rank of the tensor
t2[1,1,1].assign(33) #assign new value to the entry 1,1,1
print(t2.shape) #print shape of tensor
t0,t1,t2

tf.Tensor(3, shape=(), dtype=int32)
(2, 2, 3)


(24,
 <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=42>,
 <tf.Variable 'Variable:0' shape=(2, 2, 3) dtype=float32, numpy=
 array([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.]],
 
        [[ 6.,  7.,  8.],
         [ 9., 33., 11.]]], dtype=float32)>)

**Reshaping a tensor**

The snippet below shows how to reshape a tensor. We took the already defined t2 tensor. In the first line we reshape it to a 2-form (rank 2 tensor), which can be visualized as a matrix with 2 rows and 6 columns. In the second line we did something similar.

In [0]:
#reshaping a tensor
r1 = tf.reshape(t2,[2,6]) #2 rows 6 columns
r2 = tf.reshape(t2,[1,12]) #1 rows 12 cols
r1,r2


(<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
 array([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 33., 11.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 12), dtype=float32, numpy=
 array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 33., 11.]],
       dtype=float32)>)

In [0]:
#Tensor as a numpy/python variable
print(t2.numpy())
print(t2[1,0,2].numpy())
#number of elements of a tensor (size)
s = tf.size(input = t2).numpy()
s

[[[ 0.  1.  2.]
  [ 3.  4.  5.]]

 [[ 6.  7.  8.]
  [ 9. 33. 11.]]]
8.0


12

**Operations between tensors.**

We can operate between the elements of a tensor using the operators +, -, * and /. They also accept broadcasting, that is, arithmetic operations between tensors of different shape (just like broadcasting in python/numpy between arrays of different shape). In the example below, we multiply a rank3 tensor by a scalar

In [0]:
#Tensor operations
t2*t2
#broadcasting
t4 = t2*4
print(t4)

tf.Tensor(
[[[  0.   4.   8.]
  [ 12.  16.  20.]]

 [[ 24.  28.  32.]
  [ 36. 132.  44.]]], shape=(2, 2, 3), dtype=float32)


**Matrix multiplication and transpose**

We use tf.matmul() to multiply matrices, and tf.trnaspose() to obtain the corresponding trnaspose matrix. Here, we define two row matrices u, v, then we consider the transpose of v. Finally, we multiply v*v^t. The result is a 1x1 matrix.

In [0]:
#matrix multiplication
u = tf.constant([[3,4,5]]) #row matrix
v = tf.constant([[1,2,1]]) #row matrix
tf.matmul(u,tf.transpose(a=v)) #multiply u with v transpose

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[16]], dtype=int32)>

**Ragged tensors**

A ragged tensor has one or more ragged dimensions. A ragged dimension is a property of the slices. It basically indicates that such slices may have different shapes as seen in the example below

In [0]:
#declare ragged constant array
ragged = tf.ragged.constant([[5,2,6,1],[],[4,10,7], [8], [6,7]])
print(ragged)
print(ragged[0,:])
print(ragged[1,:])
print(ragged[2,:])
print(ragged[3,:])
print(ragged[4,:])

<tf.RaggedTensor [[5, 2, 6, 1], [], [4, 10, 7], [8], [6, 7]]>
tf.Tensor([5 2 6 1], shape=(4,), dtype=int32)
tf.Tensor([], shape=(0,), dtype=int32)
tf.Tensor([ 4 10  7], shape=(3,), dtype=int32)
tf.Tensor([8], shape=(1,), dtype=int32)
tf.Tensor([6 7], shape=(2,), dtype=int32)


**Useful tensor operations**

Difference squared (distance or norm). The exmaple below uses broadcasting.

In [0]:
x = [1 ,6, 8, 9, 4]
y = 4
s = tf.math.squared_difference(x,y)
s

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 9,  4, 16, 25,  0], dtype=int32)>

Reduce mean. A simple mean, that is, (4+5+7+3)/4=4.75

In [0]:
numbers = tf.constant([[4.,5.],[7., 3.]])
tf.reduce_mean(input_tensor=numbers)
#a simple mean

<tf.Tensor: shape=(), dtype=float32, numpy=4.75>

Mean across columns. In this case, the operation performed is (4+7)/2 = 5.5 (5+3)/2=4

In [0]:
tf.reduce_mean(input_tensor=numbers, axis=0)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([5.5, 4. ], dtype=float32)>

Mean across rows. We need to put axis = 1. The operation performed is (4+5)/2=4.5 and (7+3)/2=5

In [0]:
tf.reduce_mean(input_tensor=numbers, axis=1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([4.5, 5. ], dtype=float32)>

**Tensors with random values**

Useful when initializing weights and biases

tf.random.normal() = a tensor with values from a normal distribution

In [0]:
ran = tf.random.normal(shape=(3,2), mean=10, stddev=2, dtype=tf.float32, seed=None, name=None)
print(ran)

tf.Tensor(
[[10.2635145 10.758774 ]
 [10.161226   8.147552 ]
 [10.873544  11.679983 ]], shape=(3, 2), dtype=float32)


tf.random.uniform()= tensor filled with values from a uniform distribution in the range minval to maxval, including the lower bound but no the upper. For int32 dtype you must specify the maxval, and for dtype float32 you can write maxval = None.

In [0]:
tf.random.uniform(shape = (2,2), minval=0, maxval=12, dtype=tf.int32, seed=None, name=None)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 1,  9],
       [ 1, 11]], dtype=int32)>

**Practical example of random values**

Define two dices. Note that in shape we just write explicitly the "size" of the matrix, which is 10x1. 

In this exmaple, concatenate puts the three column matrices "one after another" to form a 10x3 matrix



In [0]:
dice1 = tf.Variable(tf.random.uniform([10,1], minval=1, maxval=7, dtype=tf.int32))
dice2 = tf.Variable(tf.random.uniform([10,1], minval=1, maxval=7, dtype=tf.int32))
#sum the two 10x1 matrices with random values
dice_sum = dice1+dice2
#to produce a single 10x3 matrix, we'll concatenate them along dim 1
result_matrix = tf.concat(values=[dice1, dice2, dice_sum], axis=1)
print(result_matrix)

tf.Tensor(
[[ 3  4  7]
 [ 3  5  8]
 [ 2  2  4]
 [ 6  6 12]
 [ 2  3  5]
 [ 4  1  5]
 [ 2  2  4]
 [ 6  4 10]
 [ 5  1  6]
 [ 5  3  8]], shape=(10, 3), dtype=int32)


**Find the indices of the elements with the largest and smallest values**

In [0]:
#1-D tensor
t5 = tf.constant([2, 5, 7, 87, 11, 45, -9, 7, -4])
print(t5)
i = tf.argmax(input=t5)
#print the index of the entry with largest value
print('index of max:',i)
#print the largest value
print('Max element:',t5[i].numpy())

tf.Tensor([ 2  5  7 87 11 45 -9  7 -4], shape=(9,), dtype=int32)
index of max; tf.Tensor(3, shape=(), dtype=int64)
Max element: 87


In [0]:
#1-D tensor
t5 = tf.constant([2, 5, 7, 87, 11, 45, -9, 7, -4])
print(t5)
i = tf.argmin(input=t5)
#print the index of the entry with largest value
print('index of min:',i)
#print the smallest value
print('min element:',t5[i].numpy())

tf.Tensor([ 2  5  7 87 11 45 -9  7 -4], shape=(9,), dtype=int32)
index of min; tf.Tensor(6, shape=(), dtype=int64)
min element: -9


In [0]:
#We'll reshape the tensor
t6 = tf.reshape(t5, [3,3]) #new tensor is a 3x3 matrix
print(t6)
i = tf.argmax(input=t6)
print('indices of max down rows:', i)
i = tf.argmin(input=t6)
print('indices of min down rows:', i)

tf.Tensor(
[[ 2  5  7]
 [87 11 45]
 [-9  7 -4]], shape=(3, 3), dtype=int32)
indices of max down rows: tf.Tensor([1 1 1], shape=(3,), dtype=int64)
indices of min down rows: tf.Tensor([2 0 2], shape=(3,), dtype=int64)


**Checkpoints**

Use checkpoints to save and restore the values of tensors


In [0]:
tvar = tf.Variable([[1,6,7,8],[5,6,2,9]])
checkpoint = tf.train.Checkpoint(var = tvar)
save_path = checkpoint.save('./vars') #save tvar
tvar.assign([[0,0,0,0],[0,0,0,1]])
tvar #print tvar with the new values
checkpoint.restore(save_path) #restore the values from checkpoint
print(tvar) #print the values of the tensor tvar (1st line)


<tf.Variable 'Variable:0' shape=(2, 4) dtype=int32, numpy=
array([[1, 6, 7, 8],
       [5, 6, 2, 9]], dtype=int32)>


**tf.function**

Takes a Python function and returns a TensorFlow graph

In [0]:
def f1(x,y):
  return tf.reduce_mean(input_tensor=tf.multiply(x**2,5)+y**2)
  f2=tf.function(f1)
  #f1 and f2 are the same thin, but f2 executes as a tensorflow graph
  x = tf.constant([4.,-5.])
  y = tf.constant([2.,3.])

  assert f1(x,y).numpy() == f2(x,y).numpy()
  #the assert states a condition, if it is true, then it does nothing

